In [ ]:
2619cb1052a25e553478de5c60d2a13f

In [6]:
pip install citipy

  Created wheel for citipy: filename=citipy-0.0.5-py3-none-any.whl size=559700 sha256=56e1febb7f84d5aa6cf089ced510b8d2a6ffce2d26c0b5c499ec2c382a0abac7
  Stored in directory: c:\users\mdebo\appdata\local\pip\cache\wheels\6d\df\5e\ad8eb9cc5ee7f4ba76865167c09f9a7edff405c669111d8353
Successfully built citipy
Note: you may need to restart the kernel to use updated packages.


In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api
#from config import g_key
from citipy import citipy

In [17]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)

In [18]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [19]:
#check length of cities
len(list(df_city["City"].unique()))

113

In [20]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['gatesville',
 'kerrville',
 'la+marque',
 'brenham',
 'port+arthur',
 'vidor',
 'santa+fe',
 'brownwood',
 'hondo',
 'lumberton',
 'san+angelo',
 'lockhart',
 'fredericksburg',
 'round+rock',
 'georgetown',
 'katy',
 'uvalde',
 'palestine',
 'nacogdoches',
 'el+campo',
 'houston',
 'texas+city',
 'taylor',
 'conroe',
 'west+university+place',
 'galveston',
 'lufkin',
 'rosenberg',
 'freeport',
 'natchitoches',
 'liberty',
 'canyon+lake',
 'angleton',
 'pleasanton',
 'bay+city',
 'victoria',
 'belton',
 'college+station',
 'austin',
 'mission+bend',
 'pflugerville',
 'robinson',
 'sulphur',
 'copperas+cove',
 'kyle',
 'bryan',
 'orange',
 'baytown',
 'huntsville',
 'seguin',
 'corsicana',
 'bridge+city',
 'tomball',
 'beeville',
 'dickinson',
 'atascocita',
 'wharton',
 'sugar+land',
 'leander',
 'beaumont',
 'woodway',
 'spring',
 'temple',
 'lakeway',
 'bellmead',
 'leon+valley',
 'clute',
 'del+rio',
 'seabrook',
 'deer+park',
 'jacksonville',
 'stafford',
 'cedar+park',
 'port+lav

In [21]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)


In [22]:
base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
temp=[]
wind_speed=[]
pressure=[]
city_name=[]

for city in target_cities[:4]:
    query_url=f"{base_current_url}appid={weather_api_key}&q={city},US&units=imperial"
    req = requests.get(query_url)
    response1=req.json()
    latitude=response1["coord"]["lat"]
    longitude=response1["coord"]["lon"]

    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        temp.append(hist_json["current"]["temp"])
        pressure.append(hist_json["current"]["pressure"])
        wind_speed.append(hist_json["current"]["wind_speed"])
        city_name.append(response1['name'])
        print(historical_url)

NameError: name 'weather_api_key' is not defined

In [23]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    city_name=[]
    country_code=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},US-TX,US&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
            
            
        except:
            print("City not found. Skipping...")
            pass
        for x in unix_time():
            historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
            hist_json = requests.get(historical_url).json()
            temp.append(hist_json["current"]["temp"])
            pressure.append(hist_json["current"]["pressure"])
            wind_speed.append(hist_json["current"]["wind_speed"])
            city_name.append(response1['name'])
            country_code.append(response1["sys"]["country"])
    d=dict()
    d["City Name"] = city_name
    d["Temperature"] = temp
    d["Wind_Speed"] = wind_speed
    d["Pressure"] = pressure    
    return(d)
        #print ("--"*30)


In [24]:
df =pd.DataFrame(find_location(target_cities))


City not found. Skipping...


UnboundLocalError: local variable 'latitude' referenced before assignment

In [ ]:
df